In [3]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [17]:
res = chat(messages)
print(res.content)

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Love is a complex and multifaceted emotion that can be experienced in many different ways. It involves feelings of deep affection, care, and attachment towards another person. Love can manifest in various forms, such as romantic love, platonic love, familial love, and self-love. It often involves a strong emotional connection, empathy, and a desire to support and nurture the well-being of the person you love. Love can bring joy, fulfillment, and a sense of belonging, but it can also be challenging and require effort and compromise to maintain. Ultimately, love is a deeply personal and subjective experience that can have a profound impact on our lives.


In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

In [22]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What does page 69 of Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition by Simon Buxton talk about?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [23]:
print(res.content)

I don't have access to specific pages or content from the book "Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition" by Simon Buxton. However, I can provide a general overview of the topics covered in the book.

The book is a practical guide that focuses on extending and customizing Microsoft Dynamics 365 Finance and Supply Chain Management using various tools and techniques. It covers a wide range of topics related to development, customization, and integration with the Dynamics 365 platform.

Some of the key topics covered in the book may include:

- Customizing forms, tables, and reports in Dynamics 365 Finance and Supply Chain Management
- Extending functionality using X++, C#, and .NET technologies
- Integrating with external systems and services
- Implementing business logic and workflows
- Enhancing security and data management
- Performance optimization and best practices for development

If you are looking for specific information from 

In [5]:
from pinecone import Pinecone

import re
import pdfplumber
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("xppcoder")

In [6]:
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL = "text-embedding-ada-002"

In [10]:
import re
from tqdm import tqdm  # Import tqdm for progress tracking

# Define a function to preprocess text
def preprocess_text(text):
    # Replace consecutive spaces, newlines, and tabs
    text = re.sub(r'\s+', ' ', text)
    return text

def process_pdf(file_path):
    # create a loader
    loader = PyPDFLoader(file_path)
    # load your data
    data = loader.load()
    # Split your data up into smaller documents with Chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(data)
    # Convert Document objects into strings with progress tracking
    texts = [str(doc) for doc in tqdm(documents, desc="Processing documents")]
    return texts

# Define a function to create embedding
# Process a PDF and create embeddings
file_path = "fnobook.pdf"  # Replace with your actual file path
texts = process_pdf(file_path)


Processing documents: 100%|██████████| 1170/1170 [00:00<00:00, 72905.40it/s]


In [11]:
def create_embeddings(texts):
    embeddings_list = []
    for text in texts:
        response = openai.embeddings.create(input=text, model=MODEL)
        embedding = response.data[0].embedding  # Access the embedding correctly
        embeddings_list.append(embedding)
    return embeddings_list

# Define a function to upsert embeddings to Pinecone with metadata
def upsert_embeddings_to_pinecone(index, embeddings, ids, texts, batch_size=100):
    for i in range(0, len(embeddings), batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        batch_texts = texts[i:i + batch_size]
        metadata = [{'text': preprocess_text(text)} for text in batch_texts]
        index.upsert(vectors=[(id, embedding, meta) for id, embedding, meta in zip(batch_ids, batch_embeddings, metadata)])


embeddings = create_embeddings(texts)

# Assuming that `file_path` is used as the ID for the entire document; if you want to use unique IDs for each text chunk, modify this accordingly
ids = [f"{file_path}_chunk_{i}" for i in range(len(embeddings))]

# Upsert the embeddings to Pinecone with text as metadata
upsert_embeddings_to_pinecone(index, embeddings, ids, texts)

In [66]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

text_field = "text"  # the metadata field that contains our text
embed_model = OpenAIEmbeddings(model=MODEL)
# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [72]:

query = "How can I add a new field to a form in Dynamics 365 using Chain of Command??"

vectorstore.similarity_search(query, k=3)

[Document(page_content="page_content='Application Extensibility Chapter 5\\n[ 184 ]Using Chain of Command with a form\\nThe forms in SCM can contain a lot of code that we may wish to augment to suit a specific\\nrequirement. We have added a service ID field to the sales order, so it may seem a fair\\nrequirement to default the service ID if it were called from a service order.\\nWhen a form is opened, an args object is passed. If we add the SalesTable menu item to\\nour service order form, and set the dataset to the service order, the args object will be\\nconstructed with a reference to this record. We can use this to filter a list of sales orders or\\nuse it in the form to set defaults on a new sales order record.\\nHow to do it...\\nFirst, we will add the sales order menu item to our service order, and then use the Chain of\\nCommand to integrate it into our solution. To add the menu item, follow these steps:\\nOpen the ConVMSVehicleServiceTable form. 1.\\nRight-click on the HeaderA

In [73]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [74]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

Based on the provided contexts, you can add a new field to a form in Dynamics 365 using Chain of Command by following these steps:

1. Identify the form: Locate the form where you want to add the new field. In this case, let's assume you want to add a field to the SalesTable form.

2. Create a form extension: In the Application Explorer, right-click on the SalesTable form and choose "Create extension." This will add a new form extension to your project.

3. Rename the form extension: Locate the newly created form extension in your project and rename it to ensure it is globally unique. For example, you can name it SalesTable.YourProjectName.

4. Open the form extension in the designer: Open the form extension you created in the designer to start customizing the form.

5. Add the new field: In the form extension designer, you can drag and drop the new field onto the form. You can also add field groups or extension fields as needed.

6. Use Chain of Command: To integrate the new field int

In [75]:
prompt = HumanMessage(
    content="Can you elaborate on using Chain of command and provide me the steps to add a new field to a form in Dynamics 365?"
)

res = chat(messages + [prompt])
print(res.content)

Using Chain of Command in Dynamics 365 allows you to extend standard objects without modifying their base code. This approach helps maintain the integrity of the original code and simplifies future upgrades. Here are the steps to add a new field to a form in Dynamics 365 using Chain of Command:

1. Identify the Form: Determine the standard form in Dynamics 365 where you want to add the new field.

2. Create an Extension Class: Create a new class that extends the standard form. This class will contain the logic for adding the new field.

3. Override the appropriate method: Identify the method in the standard form where you want to add the new field. Common methods to override include init, modifiedField, or build.

4. Add the Field in the Overridden Method: Within the overridden method, use Chain of Command APIs to add the new field to the form. Define the field properties such as data type, label, and validation rules.

5. Compile and Deploy the Extension: Compile your code, deploy the